"하나 하나 모델을 만들기보다, 미리 모델별로 만들어놓고,
모델별로 비교할 수 없을까? 가장 좋은 답변을 얻는 것으로 선택하면 되지 않는가?"
- 주의 : 여기서 말하는 모델은 '알고리즘'이 아니라 하이퍼 파라매터이다.

훈련 세트: 60%, 검증 세트: 20%, 테스트 세트: 20%
이렇게 구성하면 모델은 테스트 세트가 없더라도 기대값을 구축할 수 있습니다.
훈련 데이터가 아주 많다면 몇 %로도 전체 데이터를 대표할 수 있습니다.

In [1]:
# 검증 세트를 생성
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# test_size로 0.2, 즉 20%를 테스트 데이터로 분류한다.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# test_size로 0.2, 즉 20%를 검증 데이터로 분류한다.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

# 테스트
print(dt.score(sub_input, sub_target))

# 검증 
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


하지만 검증 데이터가 편향되거나, 학습데이터가 줄어 문제가 될 수 있다.
이럴 때는 3-폴드 교차 검증을 하면 보완이 된다.

3/1로 데이터를 나누고, A, B, C 이렇게 3등분한 후, 목적을 변경하는 것

1. A, B : 훈련, C : 검증
2. A, C : 훈련, B : 검증
3. B, C : 훈련, A : 검증

물론 3등분이 아니라 5 등분, 10등분도 가능하며, cross_validate라는 함수를 사용한다.

In [4]:

# cross_validae는 기본적으로 5등분을 한다.
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03991556, 0.02719712, 0.03321028, 0.041008  , 0.03126287]), 'score_time': array([0.00125289, 0.0011301 , 0.00175476, 0.00114799, 0.00925684]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [5]:
import numpy as  np

# 검증 폴드의 점수
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
# cross_validae는 기본적으로 5등분을 한다.
# 훈련 세트를 섞기 위해서 StratifiedKFold 를 사용한다.
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
# cross_validae는 기본적으로 5등분을 한다.
# 훈련 세트를 섞기 위해서 StratifiedKFold 를 사용한다.
# 10등분을 사용하기 위해서 n_splits
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
# 하이퍼 파라미터 : 사용자가 직접 지정해야 하는 파라미터
# 하이퍼 파라미터를 최적의 값으로 튜닝할 수 있다!
# GridSearch 클래스를 사용하여, 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행합니다.

from sklearn.model_selection import GridSearchCV

# 교차검증할 매개변수는 min_impurity_decrease 이며, 0.0001을 비롯하여 5건의 파라미터를 테스트
# n_jobs : 병렬 실행에 사용할 CPU의 코어 수 , -1인 경우, 시스템의 모든 코어를 사용합니다. 
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [12]:
# 모르는 것 있으면 질문 많이 할 것... 여기가 가장 키 포인트네.
# 1. 질문 1: 전체 훈련 세트를 사용한다고 했는데, 사용하는 모델이 무슨 모델 들인가요?
# GridSearchCV(DecisionTreeClassifier()에서 Tree 구조임을 알 수 있다.
# 책에서 말하는 모델과, 알고리즘을 혼용해서 이야기하는데, 주의할 것
# 정확하게 말하면 1개의 알고리즘에서 여러개의 하이퍼파라미터를 테스트하면서 찾는 것이다.

# 가장 좋은 훈련 점수
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
# print(dt.score(test_input, test_target))
# print(dt.score(sub_input, sub_target))
# print(dt.score(val_input, val_target))

# 가장 최적의 매개변수는 min_impurity_decrease 이며, 0.0001이 가장 좋은 값이다.
print(gs.best_params_)

# 5번의 교차 검증으로 얻은 테스트 점수 출력
print(gs.cv_results_['mean_test_score'])

# argmax() 함수를 사용하면 가장 큰 값의 인덱스를 구할 수 있다.
# 이 값이 최상의 검증 점수를 만든 매개변수 조합
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
0.8653846153846154
0.9600673562665384
0.9673076923076923
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [ ]:
# min_impurity_decrease : 불순도 감소 최소량 지정
# max_depth : 트리의 깊이 제한
# min_samples_split : 노드를 나누기 위한 최소 샘플 수

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),    # 0.0001만큼 증가하며, 0.0001 ~ 0.0009까지 (0.001은 포함안됨)
          'max_depth': range(5, 20, 1),                                 # 5에서 20까지 1만큼 증가한 배열 (20은 제외한 정수 값)
          'min_samples_split': range(2, 100, 10)}                       # 2에서 100까지 10만큼 증가한 배열 (100은 제외한 값)

print(np.arange(0.0001, 0.001, 0.0001))
print(list(range(5, 20, 1)))
print(list(range(2, 100, 10)))

[0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[2, 12, 22, 32, 42, 52, 62, 72, 82, 92]


In [ ]:
# cv값은 default로 min_impurity_decrease값마다 5건의 5-폴드 교차 검증을 한다.
# 따라서 9 * 15 * 10 * 5 = 만들어지는 모델 수가 6750개나 된다. 
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)


0.892053107562055


In [ ]:
# 최상의 매개변수 조합
print(gs.best_params_)

# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

# 가장 좋은 훈련 점수
dt = gs.best_estimator_
print(dt.score(train_input, train_target))


{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731
0.892053107562055


랜덤 서치(Random Search) :
    
    매개변수 값의 목록을 전달하는 것이 아니라, 매개 변수를 샘플링할 수 잇는 확률 분포 객체를 전달합니다

확률 분포(probability distribution) :
    
    확률 변수(random variable)가 특정한 값을 가질 확률을 나타내는 함수

In [ ]:
# 주어진 범위에서 고르게 값을 뽑으며 이것을 '균등 분포에서 샘플링한다.'라고 합니다.
# randint는 정수값을 뽑아내고, uniform은 실수값을 뽑는다.
from scipy.stats import uniform, randint

# 10개의 정수를 샘플링한다.
# rvs(random variable sampling)는 샘플링 횟수이다. 
rgen = randint(0, 10)   # 0, 1, 2, 3, ... , 9에서 값을 뽑아온다.
print(rgen.rvs(10)) # 10개를 샘플링한다.

# 1000개를 샘플링한다.
np.unique(rgen.rvs(1000), return_counts=True)



[6 9 4 5 9 0 4 8 9 4]


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97, 123,  76,  91, 100,  98, 102, 109,  93, 111]))

In [ ]:
# 실수를 샘플링한다.
# 0 ~ 1까지 10개의 샘플링을 한다.
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.15497792, 0.2094355 , 0.61495123, 0.19714651, 0.33414859,
       0.55077452, 0.52850318, 0.0202418 , 0.23093209, 0.34048124])

In [ ]:
# min_impurity_decrease : 불순도 감소 최소량 지정
# max_depth : 트리의 깊이 제한
# min_samples_split : 노드를 나누기 위한 최소 샘플 수
# min_samples_leaf : 리프노드가 되기 위한 최소 샘플의 개수, 노드가 분할할 때, 생성되는 자식의 노드 샘플 값이 이보다 작으면 분할하지 않는다.
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),                      
          'min_samples_leaf' : randint(1, 25),
          }


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa370dba490>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa370dbadd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa370dba290>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa370dba990>},
                   random_state=42)

In [ ]:
# 최적의 매개변수 조합
# {'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13)
print(gs.best_params_)

# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

# 최적의 하이퍼 파라매터로 테스트 세트의 성능을 확인해보자.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))


{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
